<a href="https://colab.research.google.com/github/WahyuKhairi06/TextMining_WahyuKhairi_2311531009/blob/main/Pertemuan%2012/TextMining_Kel3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TEXT MINING**

SCRAPING DATA TOKPED, TOKO : ASUS INDONESIA OFFICIAL. 200 HALAMAN REVIEW DARI 995 HALAMAN. DATA YANG DIAMBIL NAMA PRODUCT, NAMA, RATING, DAN ULASAN

KODE DIBAWAH HANYA BISA DIJALANKAN DARI VSCODE, JADI KALAU INGIN MENJALANKAN COPY KE VSCODE, ATUR LINK TOKPED, DAN JUMLAH HALAMAN YANG INGIN DI SCRAPING

In [ ]:
# from bs4 import BeautifulSoup
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time
# import pandas as pd

# url = input("Masukkan url toko : ")

# if url:
#     options = webdriver.ChromeOptions()
#     options.add_argument("--start-maximized")
#     driver = webdriver.Chrome(options=options)
#     driver.get(url)

#     data = []
#     for i in range(0, 200): # jumlah halaman yang diambil

#         soup = BeautifulSoup(driver.page_source, "html.parser")
#         containers = soup.find_all("article", attrs={"class": "css-1pr2lii"})

#         for container in containers:
#             try:
#                 # Ambil nama produk
#                 title_tag = container.find("p", class_="css-akhxpb-unf-heading")
#                 title = title_tag.text.strip() if title_tag else "Tidak ditemukan"

#                 rating_div = container.find("div", attrs={"data-testid": "icnStarRating"})
#                 rating = rating_div["aria-label"] if rating_div else "Tidak ada rating"

#                 date_tag = container.find("p", class_="css-1rpz5os-unf-heading")
#                 date = date_tag.text.strip() if date_tag else "Tidak ada tanggal"

#                 # Ambil ulasan
#                 review_tag = container.find("span", attrs={"data-testid": "lblItemUlasan"})
#                 review = review_tag.text.strip() if review_tag else "Tidak ditemukan"



#                 data.append((title, rating, date, review))

#             except Exception:
#                 continue

#         time.sleep(2)
#         driver.find_element(By.CSS_SELECTOR, "button[aria-label^='Laman berikutnya']").click()
#         time.sleep(3)

#     df = pd.DataFrame(data, columns=["Nama Produk", "Rating", "Tanggal", "Ulasan"])
#     df.to_csv("Tokopedia.csv", index=False)

#     print("Selesai! Data tersimpan.")


# ============================================================
# IMPORT LIBRARY
# ============================================================

In [ ]:
# Install Sastrawi if not already installed
!pip install sastrawi
!pip install gensim
!pip install emoji

import pandas as pd
import numpy as np
import re
import string
import emoji

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import gensim
from gensim import corpora


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 15.3 MB/s eta 0:00:00


# ============================================================
# LOAD DATASET
# ============================================================

In [ ]:
url = "https://raw.githubusercontent.com/ReynardGhazyTsaqif/KumpulanDataset/refs/heads/main/Tokopedia.csv"
df = pd.read_csv(url)

print("Preview Data:")
display(df.head())

print("\nInformasi Struktur Dataset:")
print(df.info())

print("\nJumlah Data:", len(df))



Preview Data:


,Nama Produk,Rating,Tanggal,Ulasan
0,Tidak ditemukan,bintang 5,Hari ini,"kualitas barang ok, diskon mantap"
1,[FREE TWS] ASUS TUF Gaming A15 FA506NCG-R735B1...,bintang 5,1 hari lalu,"Pengiriman cepat, barang sesuai spesifikasi, p..."
2,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,bintang 5,2 hari lalu,"Setelah dicoba ternyata resoonsif banget, load..."
3,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,bintang 5,2 hari lalu,barang sampai dengan amann
4,ASUS Vivobook Go 15 E1504FA-VIPS3151M - Mixed ...,bintang 5,3 hari lalu,psanan tba ssuai pking rpih mlus dn NB nya jga...



Informasi Struktur Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nama Produk  2000 non-null   object
 1   Rating       2000 non-null   object
 2   Tanggal      2000 non-null   object
 3   Ulasan       2000 non-null   object
dtypes: object(4)
memory usage: 62.6+ KB
None

Jumlah Data: 2000


In [ ]:
print(df.shape)        # jumlah baris & kolom
print(df.columns)      # nama kolom

(2000, 4)
Index(['Nama Produk', 'Rating', 'Tanggal', 'Ulasan'], dtype='object')


In [ ]:
# Hanya ambil kolom yang dipakai
df = df[['Nama Produk', 'Ulasan']]

# Hapus missing & duplikat
df = df.dropna(subset=['Ulasan'])
df = df.drop_duplicates(subset=['Nama Produk', 'Ulasan']).reset_index(drop=True)

print("Total data setelah bersih:", len(df))
display(df.head())

Total data setelah bersih: 1853


,Nama Produk,Ulasan
0,Tidak ditemukan,"kualitas barang ok, diskon mantap"
1,[FREE TWS] ASUS TUF Gaming A15 FA506NCG-R735B1...,"Pengiriman cepat, barang sesuai spesifikasi, p..."
2,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,"Setelah dicoba ternyata resoonsif banget, load..."
3,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,barang sampai dengan amann
4,ASUS Vivobook Go 15 E1504FA-VIPS3151M - Mixed ...,psanan tba ssuai pking rpih mlus dn NB nya jga...


NAMA PRODUK SEBAGIAN BESAR TIDAK BISA DI SCRAPING DARI TOKPED, HANYA ULASAN YANG BISA, SEHINGGA NAMA PORDUK MENJADI "TIDAK DITEMUKAN"

In [ ]:
jumlah_tidak_ditemukan = (df['Nama Produk'] == "Tidak ditemukan").sum()
print("Jumlah review tanpa nama produk:", jumlah_tidak_ditemukan)

Jumlah review tanpa nama produk: 1535


# ============================================================
# TEXT PREPROCESSING
# ============================================================

In [ ]:
def clean_text(text):

    text = text.lower()

    # Hapus URL
    text = re.sub(r"http\S+|www.\S+", "", text)

    # Hapus HTML tag
    text = re.sub(r"<.*?>", "", text)

    # Hapus emoji
    text = emoji.replace_emoji(text, replace="")

    # Hapus angka
    text = re.sub(r"\d+", "", text)

    # Hapus repeated letters "baguuuuus" → "bagus"
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Hapus punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Hapus spasi berlebihan
    text = re.sub(r"\s+", " ", text).strip()

    return text

df['clean_text'] = df['Ulasan'].astype(str).apply(clean_text)


In [ ]:
def tokenize(text):
    return text.split()

df['tokens'] = df['clean_text'].apply(tokenize)


In [ ]:
stop_factory = StopWordRemoverFactory()
stopwords = set(stop_factory.get_stop_words())

extra_stopwords = {
    "nya", "nih", "dong", "sih", "ga", "gak", "kok", "ya", "iyaa", "lho",
    "haha", "hehe", "wkwk", "oke", "ok", "sip", "banget", "bgt",
    "aku", "kamu", "dia", "kami", "kita", "saya",
    "barang", "produk", "pesanan", "toko", "seller", "blm", "udh", "udah","dan","juga"
}

stopwords.update(extra_stopwords)

def remove_stopwords(tokens):
    return [t for t in tokens if t not in stopwords and len(t) > 2]

df['tokens'] = df['tokens'].apply(remove_stopwords)


In [ ]:
slang_dict = {
    "bgs":"bagus",
    "bgus":"bagus",
    "bgsnya":"bagus",
    "mantappp":"mantap",
    "mantapp":"mantap",
    "mantabb":"mantap",
    "rekomen":"rekomendasi",
    "rekom":"rekomendasi",
    "rekomened":"rekomendasi",
    "mlus":"mulus",
    "mlusnya":"mulus",
    "hlus":"mulus",
    "lus":"mulus",
    "psanan":"pesanan",
    "tba":"tiba",
    "bntu":"bantu",
    "krm":"kirim",
    "ksh":"kasih",
    "bgt":"banget",
    "seger":"segar",
    "pking":"paking",
    "rpih":"rapi"
}

def normalize_slang(tokens):
    return [slang_dict.get(t, t) for t in tokens]

df['tokens'] = df['tokens'].apply(normalize_slang)


In [ ]:
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def stemming(tokens):
    return [stemmer.stem(t) for t in tokens]

df['tokens'] = df['tokens'].apply(stemming)


In [ ]:
df['tokens'] = df['tokens'].apply(lambda x: [t for t in x if len(t) > 2])

df = df[df['tokens'].map(len) > 0].reset_index(drop=True)


In [ ]:
dictionary = corpora.Dictionary(df['tokens'])
dictionary.filter_extremes(no_below=3, no_above=0.5)

corpus = [dictionary.doc2bow(t) for t in df['tokens']]

print("Total Dokumen:", len(corpus))
print("Total Vocabulary:", len(dictionary))

df[['Nama Produk', 'Ulasan', 'tokens']].head(20)


Total Dokumen: 1829
Total Vocabulary: 590


,Nama Produk,Ulasan,tokens
0,Tidak ditemukan,"kualitas barang ok, diskon mantap","[kualitas, diskon, mantap]"
1,[FREE TWS] ASUS TUF Gaming A15 FA506NCG-R735B1...,"Pengiriman cepat, barang sesuai spesifikasi, p...","[kirim, cepat, sesuai, spesifikasi, packingan,..."
2,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,"Setelah dicoba ternyata resoonsif banget, load...","[coba, nyata, resoonsif, loading, cepat, kalau..."
3,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,barang sampai dengan amann,[amann]
4,ASUS Vivobook Go 15 E1504FA-VIPS3151M - Mixed ...,psanan tba ssuai pking rpih mlus dn NB nya jga...,"[pesan, tiba, ssuai, paking, rapi, mulus, jga,..."
5,[FREE SMARTWATCH] ASUS Vivobook 14 A1405VA-VIP...,Aman sampai tujuann,"[aman, tujuann]"
6,ASUS Vivobook 14 A1404VA-FHD3821M - Quiet Blue...,"Packing mantap, barang aman, lisensi OS dan of...","[packing, mantap, aman, lisensi, office, inclu..."
7,Tidak ditemukan,packing mantap,"[packing, mantap]"
8,[FREE MOUSE] ASUS Vivobook 14 A1404ZA-VIPS551 ...,"alhamdulillah udah bisa kerja lg, tiba cepat d...","[alhamdulillah, kerja, tiba, cepat, mulus, ses..."
9,[FREE MOUSE] ASUS Vivobook Go 15 E1504FA-VIPS5...,Pengiriman sangat cepat dan barang diterima de...,"[kirim, sangat, cepat, terima, baik, sangat, b..."


# LDA

In [ ]:
from gensim.models import LdaModel, CoherenceModel

scores = []

print("Menghitung Coherence Score...")

for k in range(2, 9):  # coba 2–8 topik
    lda_test = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=k,
        passes=15,
        random_state=42
    )

    coherence = CoherenceModel(
        model=lda_test,
        texts=df["tokens"],
        dictionary=dictionary,
        coherence='c_v'
    ).get_coherence()

    scores.append((k, coherence))
    print(f"K = {k} | Coherence = {coherence:.4f}")

Menghitung Coherence Score...
K = 2 | Coherence = 0.3200
K = 3 | Coherence = 0.3427
K = 4 | Coherence = 0.3353
K = 5 | Coherence = 0.3408
K = 6 | Coherence = 0.3618
K = 7 | Coherence = 0.3295
K = 8 | Coherence = 0.3097


In [ ]:
K = 6

lda_final = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=K,
    passes=20,
    random_state=42
)

In [ ]:
topics = lda_final.print_topics(num_words=12)
for topic in topics:
    print(topic)

(0, '0.081*"moga" + 0.076*"awet" + 0.067*"bagus" + 0.040*"laptop" + 0.021*"original" + 0.018*"kualitas" + 0.016*"mantap" + 0.015*"kerja" + 0.015*"mouse" + 0.014*"keren" + 0.014*"asus" + 0.014*"buat"')
(1, '0.077*"aman" + 0.059*"kirim" + 0.049*"packing" + 0.034*"sesuai" + 0.033*"bagus" + 0.032*"cepat" + 0.027*"laptop" + 0.019*"kayu" + 0.019*"rapi" + 0.015*"selamat" + 0.013*"rapih" + 0.012*"lama"')
(2, '0.116*"baik" + 0.049*"fungsi" + 0.042*"kirim" + 0.040*"aman" + 0.040*"terima" + 0.035*"cepat" + 0.024*"kondisi" + 0.019*"laptop" + 0.017*"sesuai" + 0.015*"bagus" + 0.015*"sangat" + 0.014*"packing"')
(3, '0.075*"sesuai" + 0.053*"terima" + 0.050*"sangat" + 0.043*"kasih" + 0.030*"beli" + 0.024*"deskripsi" + 0.022*"puas" + 0.021*"bagus" + 0.018*"asus" + 0.016*"terimakasih" + 0.016*"sekali" + 0.015*"baik"')
(4, '0.082*"cepat" + 0.062*"kirim" + 0.059*"sesuai" + 0.056*"mantap" + 0.051*"respon" + 0.034*"fast" + 0.031*"admin" + 0.024*"deskripsi" + 0.023*"bonus" + 0.021*"layan" + 0.017*"ori" + 0.01

In [ ]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.5 MB/s eta 0:00:00


In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_final, corpus, dictionary)
vis

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.054645 -0.101383       1        1  20.000289
3      0.191663  0.073993       2        1  18.811923
2      0.031460 -0.076966       3        1  17.231548
4      0.115715 -0.030700       4        1  17.197344
0     -0.123798  0.216258       5        1  14.909602
5     -0.160395 -0.081202       6        1  11.849295, topic_info=         Term        Freq       Total Category  logprob  loglift
29       baik  303.000000  303.000000  Default  30.0000  30.0000
42       moga  153.000000  153.000000  Default  29.0000  29.0000
4        awet  140.000000  140.000000  Default  28.0000  28.0000
39     terima  207.000000  207.000000  Default  27.0000  27.0000
6       kirim  369.000000  369.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
99     ringan   11.417352   35.477070   Topic6  -4.8374   0.9991
46      bagus   21.575493  336.767750   Topic6  -4.2010  -0.6149
115    respon   15.827599  170.480252   Topic6  -4.5108  -0.2440
31    packing   14.125077  182.428332   Topic6  -4.6246  -0.4255
1    kualitas   11.338827   93.653860   Topic6  -4.8443   0.0215

[380 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
186       5  0.786975  aamiin
114       1  0.017997   admin
114       2  0.269952   admin
114       3  0.044992   admin
114       4  0.593894   admin
...     ...       ...     ...
172       1  0.734825    wrap
172       3  0.244942    wrap
461       2  0.094672     you
461       4  0.757373     you
461       6  0.094672     you

[656 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 3, 5, 1, 6])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
